In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np

from itertools import product

In [ ]:
ticker = "JPM"
data = yf.download(tickers = ticker, start="2006-01-01", end="2025-01-01")

In [ ]:
data

In [4]:
data["Daily return"] = data["Adj Close"].pct_change() #periods=n
data["State"] = np.where(data["Daily return"] >= 0, "up", "down")

In [5]:
up_counts = len(data[data["State"] == "up"])
down_counts = len(data[data["State"] == "down"])

up_to_up = len(data[(data["State"] == "up") & (data["State"].shift(-1) == "up")])/up_counts #/len(data.query('State=="up"'))
up_to_down = len(data[(data["State"] == "up") & (data["State"].shift(-1) == "down")])/len(data[data["State"] == "up"])
down_to_up = len(data[(data["State"] == "down") & (data["State"].shift(-1) == "up")])/len(data[data["State"] == "down"])
down_to_down = len(data[(data["State"] == "down") & (data["State"].shift(-1) == "down")])/down_counts

In [6]:
transition_matrix = pd.DataFrame({
    "up": [up_to_up, down_to_up],
    "down": [up_to_down, down_to_down]
}, index=["up", "down"])

In [ ]:
print(transition_matrix)

In [8]:
condition1 = data[
    (data["State"] == "up") 
    & (data["State"].shift(1) == "down")
    & (data["State"].shift(2) == "down")
    & (data["State"].shift(3) == "down")
    & (data["State"].shift(4) == "down")
    & (data["State"].shift(5) == "down")
    ]

condition2 = data[
    (data["State"] == "down")
    & (data["State"].shift(1) == "down")
    & (data["State"].shift(2) == "down")
    & (data["State"].shift(3) == "down")
    & (data["State"].shift(4) == "down")
    ]

In [ ]:
print(f"P(up | 5 down) = {len(condition1)/ len(condition2)}")

In [ ]:
test = data
test["test"] = data["State"].shift(1)
test["test2"] = data["State"].shift(2)
test

In [15]:
sequence = ["down", "down", "down", "down", "down", "up"]

# Build the condition using a loop
condition = True
for i, state in enumerate(sequence):
    condition &= (data["State"].shift(-i) == state)

# Filter the DataFrame
filtered_data = data[condition]

In [29]:
states = ["up", "down"]
transition_probabilities = {}

for length in range(2, 11): 
    for state_comb in product(states, repeat=length):
        condition1 = True
        condition2 = True
        for i, state in enumerate(state_comb):
            # In our case it does not matter if we look forward (i.e. -i) or backwards (i.e. i) because in any case we iterate over all combinations.
            condition1 &= (data["State"].shift(i) == state)
            # The first state is the one we "predict"
            if i != 0:
                condition2 &= (data["State"].shift(i) == state)

        transition_probabilities[state_comb] = (len(data[condition1])/len(data[condition2]), len(data[condition2]))

In [ ]:
for state_comb, prob in transition_probabilities.items():
    if prob[0] > 0.7 and prob[1] > 30:
        print(f"Probability of {state_comb[0]} after {state_comb[1:]} is {prob[0]}, pattern appeared {prob[1]} times")

Exit pattern